# caf Visualiser

This notebook shows you the final output of the full workflow, and could be useful to compare with the output of visualiser.ipynb.
Right now it makes use of the flatcaf, rather than caf filetype.

Currently the notebook will grab the neutrino slice for each event, plot the true start and end points of a primary particle, and either the reconstructed track or shower (depending on whether cnnscore.track or cnnscore.shower is higher).

In [26]:
from tools import *
import uproot
import matplotlib.pyplot as plt
import awkward as ak
import plotly.io as pio
import plotly.colors as pc
pio.templates['plotly'].layout['autosize'] = False

In [27]:
file_name = 'reco2_gen_ee_lowmass_hnl.flat.caf.root'
caffile = uproot.open(file_name)
print(caffile.keys())
df = caffile['recTree'].arrays(library='pd')

['env;1', 'env/envtree;2', 'env/envtree;1', 'TotalPOT;1', 'TotalEvents;1', 'recTree;1', 'GenieEvtRecTree;1', 'metadata;1', 'metadata/metatree;1']


In [28]:
# Grabbing only the neutrino slice part of the dataframe
sub_key_list = []
for key in df.keys():
    split_key = key.split('.')

    if 'slc' == split_key[1]:
        sub_key_list.append(key)

df_red = df[sub_key_list]
df_red.columns = df_red.columns.str.replace('rec.slc.', '')

Defining the variables we want to grab & plot

In [ ]:
true_primary_vars = {
    'start': ['truth.prim.start.x', 'truth.prim.start.y', 'truth.prim.start.z'],
    'end': ['truth.prim.end.x', 'truth.prim.end.y', 'truth.prim.end.z'],
    'pdg': ['truth.prim.pdg'],
}

reco_shower_vars = {
    'start': ['reco.pfp.shw.start.x', 'reco.pfp.shw.start.y', 'reco.pfp.shw.start.z'],
    'end': ['reco.pfp.shw.end.x', 'reco.pfp.shw.end.y', 'reco.pfp.shw.end.z'],
    'angle': ['reco.pfp.shw.open_angle'],
}

reco_track_vars = {
    'start': ['reco.pfp.trk.start.x', 'reco.pfp.trk.start.y', 'reco.pfp.trk.start.z'],
    'end': ['reco.pfp.trk.end.x', 'reco.pfp.trk.end.y', 'reco.pfp.trk.end.z'],
}

track_shower_scores = ['reco.pfp.cnnscore.track', 'reco.pfp.cnnscore.shower', 'reco.pfp.trackScore']


In [ ]:

N_events = 3 # len(df_red.index)

fig = go.Figure()
fig = make_detector_box(fig)

fig.add_trace(go.Scatter3d(
    x=[-10000, -10000],
    y=[-10000, -10000],
    z=[-10000, -10000],
    mode='lines',
    line=dict(color=pc.qualitative.Dark24[-1], width=2),
    name=f'True primary',
    hoverinfo=None,
))

for i in range(N_events):
    event = df_red.iloc[i]
    
    
    true_start_all = event[true_primary_vars['start']]
    true_end_all = event[true_primary_vars['end']]
    pdg_all = event[true_primary_vars['pdg']][0]
    for primary in range(len(true_start_all[0])):
        if np.isnan(true_start_all[0][primary]) or np.isnan(true_end_all[0][primary]):
            continue  # Skip if start or end is NaN
        start = [true_start_all[0][primary], true_start_all[1][primary], true_start_all[2][primary]]
        end = [true_end_all[0][primary], true_end_all[1][primary], true_end_all[2][primary]]
        pdg = pdg_all[primary] # Get the PDG code for the primary particle

        if np.abs(pdg) in pdg_color_name_dict.keys():
            pid = pdg_color_name_dict[np.abs(pdg)][-1]
            color = pdg_color_name_dict[np.abs(pdg)][0]
        else:
            pid = f'Unknown ({pdg})'
            color = 'black'  # Default color for unknown particles

        if 'nu' in pid:
            fig.add_trace(go.Scatter3d(
                x=[start[0], end[0]],
                y=[start[2], end[2]],
                z=[start[1], end[1]],
                mode='lines',
                line=dict(dash='dash',color=color, width=2),
                name=f'Primary {pid}, Event{i}',
                hoverinfo=None,
                showlegend=False
            ))
        else:
            fig.add_trace(go.Scatter3d(
                x=[start[0], end[0]],
                y=[start[2], end[2]],
                z=[start[1], end[1]],
                mode='lines',
                line=dict(color=color, width=2),
                name=f'Primary {pid}, Event{i}',
                hoverinfo=None,
                showlegend=False
            ))


    # Looping over all reco showers in each event
    
    reco_start_all = event[reco_shower_vars['start']]
    reco_end_all = event[reco_shower_vars['end']]
    opening_angle_all = event[reco_shower_vars['angle']][0]  # Opening angle in degrees

    addLegend = True
    color = pc.qualitative.Light24[i]  # Use a different color for reco showers

    for shr in range(len(reco_start_all[0])):
        if np.isnan(reco_start_all[0][shr]) or np.isnan(reco_end_all[0][shr]):
            continue
        scores = [event[track_shower_scores[0]][shr], event[track_shower_scores[1]][shr], event[track_shower_scores[2]][shr]]

        shower_score = scores[1]
        track_score = scores[0]
        
        if track_score > shower_score:
            continue  # Skip if the track score is greater than the shower score
        
        start = [reco_start_all[0][shr], reco_start_all[1][shr], reco_start_all[2][shr]]
        end = [reco_end_all[0][shr], reco_end_all[1][shr], reco_end_all[2][shr]]
        opening_angle = opening_angle_all[shr]  # Opening angle in radians

        opening_angle = np.degrees(opening_angle)  # Convert to degrees

        fig = plot_shower_cone(fig, start, end, opening_angle, shower_name=f'Event{i}', color=color, addLegend=addLegend)
        addLegend = False

    # Looping over all reco tracks in each event
    reco_start_all = event[reco_track_vars['start']]
    reco_end_all = event[reco_track_vars['end']]

    for trk in range(len(reco_start_all[0])):
        if np.isnan(reco_start_all[0][trk]) or np.isnan(reco_end_all[0][trk]):
            continue
        scores = [event[track_shower_scores[0]][shr], event[track_shower_scores[1]][shr], event[track_shower_scores[2]][shr]]

        shower_score = scores[1] 
        track_score = scores[0] 
        if track_score < shower_score:
            continue  # Skip if the track score is less than the shower score

        start = [reco_start_all[0][trk], reco_start_all[1][trk], reco_start_all[2][trk]]
        end = [reco_end_all[0][trk], reco_end_all[1][trk], reco_end_all[2][trk]]

        fig.add_trace(go.Scatter3d(
            x=[start[0], end[0]],
            y=[start[2], end[2]],
            z=[start[1], end[1]],
            mode='lines',
            line=dict(color=pc.qualitative.Light24[i], width=2),
            name=f'Reco track Event{i}',
            hoverinfo=None,
            showlegend=False
        ))


width_x = sbnd_det_box['x'][1] - sbnd_det_box['x'][0]
width_y = sbnd_det_box['y'][1] - sbnd_det_box['y'][0]
width_z = sbnd_det_box['z'][1] - sbnd_det_box['z'][0]

x_lims = [sbnd_det_box['x'][0]-0.1*width_x, sbnd_det_box['x'][1]+0.1*width_x]
y_lims = [sbnd_det_box['y'][0]-0.1*width_y, sbnd_det_box['y'][1]+0.1*width_y]
z_lims = [sbnd_det_box['z'][0]-0.1*width_z, sbnd_det_box['z'][1]+0.1*width_z]

fig.update_scenes(xaxis_title_text='X (cm)',  
        yaxis_title_text='Z (cm)',  
        zaxis_title_text='Y (cm)',
        xaxis=dict(range=x_lims),
        yaxis=dict(range=z_lims),
        zaxis=dict(range=y_lims),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)
),

fig.layout.height = 800
fig.layout.width = 1200
fig.show()




/tmp/ipykernel_3608879/3541128311.py:41: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_3608879/3541128311.py:42: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_3608879/3541128311.py:43: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_3608879/3541128311.py:45: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated 